<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/MCA_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connecting Drive to Google Colab**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Loading, Data Preprocessing for video**

In [0]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

def processData(data):
    # print(data)
    X = data.iloc[:,:].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    X = np.mean(X, 0)
    X = np.array(X.reshape(1, -1))
    # print(X)
    # print(X.shape)
    return X

def getData(filename):
    # print(filename)
    data = pd.read_csv(filename,delimiter=',')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file1 = (patientNo)+"_CLNF_AUs.txt"
    file2 = (patientNo)+"_CLNF_features.txt"
    file3 = (patientNo)+"_CLNF_features3D.txt"
    file4 = (patientNo)+"_CLNF_gaze.txt"
    file5 = (patientNo)+"_CLNF_hog.txt"
    file6 = (patientNo)+"_CLNF_pose.txt"

    X1 = getData(file1)
    X2 = getData(file2)
    X3 = getData(file3)
    X4 = getData(file4)
    # X5 = getData(file5)
    X6 = getData(file6)

    X = np.concatenate((X1, X2, X3, X4, X6), 1)
    return X


dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
positive = []
negative = []

for i in range(len(dataset)):
    if dataset[i][0]<=310 and dataset[i][0]>=300:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

    elif dataset[i][0]<=355 and dataset[i][0]>=345:
        if(dataset[i][1] == 0):
            positive.append(int(dataset[i][0]))
        else:
            negative.append(int(dataset[i][0]))

X = np.zeros((1,388))
for i in positive:
    string = '/content/drive/My Drive/MCA Dataset/positive_data/'+str(i)
    X_temp = makeDataPoint(string)
    X = np.concatenate((X,X_temp),0)
Y = np.zeros((len(positive),1))
for i in negative:
    string = '/content/drive/My Drive/MCA Dataset/negative_data/'+str(i)
    X_temp = makeDataPoint(string)
    X = np.concatenate((X,X_temp),0)
Y = np.concatenate((Y,np.ones((len(negative),1))),0)
X = np.delete(X,0,0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:80: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

**RF and SVM on dataset**

In [0]:
print("train")
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
print("dev")
X_dev, Y_dev = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')
count0 = 0
X_train = np.concatenate((X_train, X_dev), 0  )
Y_train = np.concatenate((Y_train, Y_dev), 0  )

print("test")
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

print(Y_train)

clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
clf1.predict(X_test)
print("Random forest: "+ str(clf1.score(X_test,Y_test)))

from sklearn.svm import SVC
classifier2 = SVC(kernel = 'rbf', random_state = 0)
classifier2.fit(X_train, Y_train)
classifier2.predict(X_test)
print("SVM rbf: "+ str(classifier2.score(X_test,Y_test)))

clf1.predict(X_train)
print("Random forest: train"+ str(clf1.score(X_train,Y_train)))

classifier2.predict(X_train)
print("SVM: train  "+ str(classifier2.score(X_train,Y_train)))


from sklearn.metrics import classification_report
y_pred = classifier2.predict(X_test)
print(classification_report(Y_test, y_pred))

train
dev


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139) have mixed types. Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105

test


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139) have mixed types. Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105

[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]
Random forest: 0.5121951219512195
SVM rbf: 0.3170731707317073
Random forest: train1.0
SVM: train  0.5384615384615384
              precision    recall  f1-score   support

           0       0.60      0.10      0.18        29
           1       0.28      0.83      0.42        12

    accuracy                           0.32        41
   macro avg       0.44      0.47      0.30        41
weighted avg       0.51      0.32      0.25        41



In [0]:
import nltk

In [0]:
X_train = np.concatenate((X_train, X_dev), 0  )
Y_train = np.concatenate((Y_train, Y_dev), 0  )
count0=0
print("test")
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

print(Y_train)

clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
clf1.predict(X_test)
print("Random forest: "+ str(clf1.score(X_test,Y_test)))

from sklearn.svm import SVC
classifier2 = SVC(kernel = 'rbf', random_state = 0)
classifier2.fit(X_train, Y_train)
classifier2.predict(X_test)
print("SVM rbf: "+ str(classifier2.score(X_test,Y_test)))

clf1.predict(X_train)
print("Random forest: train"+ str(clf1.score(X_train,Y_train)))

classifier2.predict(X_train)
print("SVM: train  "+ str(classifier2.score(X_train,Y_train)))


from sklearn.metrics import classification_report
y_pred = classifier2.predict(X_test)
print(classification_report(Y_test, y_pred))